In [71]:
import csv
from urllib.request import urlopen
from bs4 import BeautifulSoup, Comment
import requests
import re
import pandas as pd
import html5lib
from selenium import webdriver
import time
from selenium.webdriver.chrome.options import Options

In [8]:
# Firstly we need to collect all links that we intend to scrape from
# the data needed is in the format of:
# https://fantasydata.com/nfl/point-spreads-and-odds?season=2011&seasontype=1&week=1
# We need to set up a dictionary containing all weeks and years: seasonDict

# First setup years and weeks we are interested in
# We will collect dating back to 2004
yearNum = [year for year in range(2011,2019)]
weekNum = [week for week in range(1,18)]

seasonDict = {i : [j for j in weekNum] for i in yearNum}
print(seasonDict)

{2011: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17], 2012: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17], 2013: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17], 2014: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17], 2015: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17], 2016: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17], 2017: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17], 2018: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]}


In [9]:
# Now we can build the links we need, start with empty list: seasonLinks
seasonLinks = []

# Now iterate through seasonDict and put together all needed links
# remeber the format of the links is: https://www.pro-football-reference.com/years/2018/week_21.htm
for year in seasonDict.keys():
    for week in seasonDict[year]:
        seasonLinks.append('https://fantasydata.com/nfl/point-spreads-and-odds?season=' + str(year) + '&seasontype=1&week=' + str(week))

print(seasonLinks[33])

https://fantasydata.com/nfl/point-spreads-and-odds?season=2012&seasontype=1&week=17


In [10]:
# Setup a csv file with column names to hold our data
csvFile = open("./csvFiles/lineData.csv", 'w+')
writer = csv.writer(csvFile)
writer.writerow(('season','week','date','favorite','spread', 'underdog', 'total', 'awayMoneyLine', 'homeMoneyLine'))


77

In [53]:
# Now that we have all the links needed to scrape game data, we will aim to
# collect the following variables from each page: date, favorite, spread,
# underdog, total, awayMoneyLine, homeMoneyLine. We will also add in season/week

url = 'https://fantasydata.com/nfl/point-spreads-and-odds?season=2015&seasontype=1&week=8'
page = requests.get(url)
soup = BeautifulSoup(page.text, "html.parser")


# First let's grab the season and week using regex
nums = re.findall(r'\d+', url)

season = nums[0]
week = nums[-1]

# now collect data from the table on the webpage
chrome_path = r"C:\Users\Michael\chromedriver.exe"
driver = webdriver.Chrome(chrome_path)
driver.implicitly_wait(5)

driver.get(url)
df = pd.read_html(driver.page_source)[1]

Int64Index([0, 1, 2, 3, 4, 5, 6], dtype='int64')

In [55]:
df.columns = ['date', 'favorite', 'spread', 'underdog', 'total', 'awayMoneyLine', 'homeMoneyLine']
df['season'] = season
df['week'] = week
df

,date,favorite,spread,underdog,total,awayMoneyLine,homeMoneyLine,season,week
0,"October 29, 2015 8:25 PM",at Patriots,-8.0,Dolphins,51.0,320,-400,2015,8
1,"November 1, 2015 9:30 AM",at Chiefs,-3.0,Lions,45.0,140,-160,2015,8
2,"November 1, 2015 1:00 PM",at Rams,-8.0,49ers,41.0,315,-400,2015,8
3,"November 1, 2015 1:00 PM",at Falcons,-8.0,Buccaneers,48.0,333,-400,2015,8
4,"November 1, 2015 1:00 PM",at Ravens,-4.0,Chargers,50.0,175,-210,2015,8
5,"November 1, 2015 1:00 PM",Vikings,-2.0,at Bears,44.0,-120,110,2015,8
6,"November 1, 2015 1:00 PM",Cardinals,-7.0,at Browns,46.0,-320,260,2015,8
7,"November 1, 2015 1:00 PM",at Saints,-3.0,Giants,52.0,145,-170,2015,8
8,"November 1, 2015 1:00 PM",at Texans,-4.0,Titans,43.0,155,-175,2015,8
9,"November 1, 2015 1:00 PM",at Steelers,-0.0,Bengals,49.0,-115,-105,2015,8


In [87]:
# Now let's put it all together in one final script

# Setup an empty DF to append to
mainDf = pd.DataFrame(columns=['date', 'favorite', 'spread', 'underdog', 'total', 'awayMoneyLine', 'homeMoneyLine','season','week'])

# Setup cols of interest w/in loop
cols = ['date', 'favorite', 'spread', 'underdog', 'total', 'awayMoneyLine', 'homeMoneyLine']

# Execute loop
for link in seasonLinks:
    
    time.sleep(60)
    
    url = link
    page = requests.get(url)
    soup = BeautifulSoup(page.text, "html.parser")
   
    nums = re.findall(r'\d+', url)

    season = nums[0]
    week = nums[-1]

    chrome_path = r"C:\Users\Michael\chromedriver.exe"
    
    options = webdriver.ChromeOptions()
    options.add_argument('headless')
    options.add_argument('window-size=1920x1080')
    options.add_argument("disable-gpu")


    driver = webdriver.Chrome(chrome_path, options=options)

    driver.get(url)
    df = pd.read_html(driver.page_source)[1]
    
    df.columns = cols
    df['season'] = season
    df['week'] = week
    
    
    # Append this table's data to the main DF
    mainDf = mainDf.append(df)

In [88]:
mainDf.tail()

,date,favorite,spread,underdog,total,awayMoneyLine,homeMoneyLine,season,week
11,"December 30, 2018 4:25 PM",at Seahawks,-15.0,Cardinals,40.0,675.0,-860.0,2018,17
12,"December 30, 2018 4:25 PM",at Chiefs,-14.0,Raiders,53.0,700.0,-900.0,2018,17
13,"December 30, 2018 4:25 PM",at Rams,-10.5,49ers,50.0,390.0,-440.0,2018,17
14,"December 30, 2018 4:25 PM",at Vikings,-5.5,Bears,39.5,215.0,-235.0,2018,17
15,"December 30, 2018 8:20 PM",Colts,-5.5,at Titans,42.5,-235.0,215.0,2018,17


In [92]:
mainDf.to_csv('C:\\Users\\Michael\\Desktop\\Springboard\\Capstone2\\csvFiles\\lineData.csv')

In [93]:
%store mainDf

Stored 'mainDf' (DataFrame)
